In [53]:
#!pip install dbdpy
#!pip install dbfread
#!pip install pandas
#!pip install bar_chart_race
#!pip install matplotlib pandas
#!pip install Flask
#!pip install plotly


In [54]:
import os
import pandas as pd
from dbfread import DBF
import textwrap


In [55]:
#Leemos la lista de RUC
df_ruc=pd.read_excel("D:/Proyectos/Ranking Palta/ranking-palta/RUCtop200.xlsx")
df_ruc["NRO_DOCU"]=df_ruc["NRO_DOCU"].astype(str)
print(df_ruc.dtypes)

NRO_DOCU             object
Nombre de empresa    object
dtype: object


In [56]:
#Especificamos la ruta de los archivos DFB
CARPETA_PALTA = "D:/Proyectos/Ranking Palta/ranking-palta/Datos"

#creamos una lista en donde se van a almacenar los dataframes
dataframes = []

for archivo in os.listdir(CARPETA_PALTA):
    if archivo.endswith(".DBF"):
        # Leemos el archivo
        rutaArchivo = os.path.join(CARPETA_PALTA, archivo)        
        tabla = DBF(rutaArchivo, encoding="latin1")
        df = pd.DataFrame(iter(tabla))
        dataframes.append(df)

# Concatenamos todos los Dataframes en uno solo
df_concatenado = pd.concat(dataframes, ignore_index=True)

# Filtramos los RUC: "No disponibles"
df_concatenado=df_concatenado[df_concatenado["NRO_DOCU"]!="No Disponi."]

# Convertimos la columna 'FECHA' a texto
df_concatenado['FECHA'] = df_concatenado['FECHA'].astype(str)

# Creamos la lista de posiciones que queremos cambiar
posiciones = [4, 6]

# Creamos una función para insertar cualquier letra en posiciones que deseamos
def insertar_letras(valor, letra, posiciones):
    for posicion in sorted(posiciones, reverse=True):
        valor = valor[:posicion] + letra + valor[posicion:]
    return valor

# Aplicamos la función a la columna del dataframe
df_concatenado['FECHA_MOD'] = df_concatenado['FECHA'].apply(lambda x: insertar_letras(x, "/", posiciones))
df_concatenado['FECHA_MOD_FER'] = pd.to_datetime(df_concatenado["FECHA_MOD"], format = "%Y/%m/%d")
df_concatenado['FOB_DOLPOL_MILL'] = df_concatenado['FOB_DOLPOL']/1000000
df_concatenado['FOB_DOLPOL_MILL'] =df_concatenado['FOB_DOLPOL_MILL'].round(2)
# Realizamos el inner join
df_concatenado0 = pd.merge(df_concatenado, df_ruc, on='NRO_DOCU', how='inner')

# Creamos una tabla dinámica de los datos que queremos
tabla_dinamica= pd.pivot_table(
    df_concatenado0,
    values=["FOB_DOLPOL_MILL"],
    index= "FECHA_MOD_FER",
    columns= "Nombre de empresa",
    aggfunc= {
        "FOB_DOLPOL_MILL": "sum"
    }

)

# Asegúrate de que df_tabla_dinamica esté en el formato correcto
df_tabla_dinamica = tabla_dinamica.fillna(0)  # Rellenar valores NaN con 0

# Aplanar el MultiIndex
df_tabla_dinamica.columns = [f'{col[1]}' for col in df_tabla_dinamica.columns]

df_tabla_dinamica=df_tabla_dinamica.cumsum()

def agregar_saltos_linea_columna(nombre_columna, max_ancho=10):
    return '\n'.join(textwrap.wrap(nombre_columna, max_ancho))

# Aplicar la función a los nombres de las columnas
df_tabla_dinamica.columns = [agregar_saltos_linea_columna(col) for col in df_tabla_dinamica.columns]

#df_tabla_dinamica=tabla_dinamica.reset_index()

#df_tabla_dinamica.drop('NRO_DOCU', axis=1, inplace=True)

# Pasamos la fecha a un index
#df_tabla_dinamica.set_index("FECHA_MOD_FER", inplace = True)
print(df_tabla_dinamica["CONSORCIO\nDE PRODUCT\nORES DE\nFRUTA S.A."].head(25))
print(df_tabla_dinamica.head(25))
print(df_tabla_dinamica.index)

FECHA_MOD_FER
2001-06-01    0.00
2001-06-04    0.00
2001-06-10    0.09
2001-06-16    0.21
2001-06-24    0.36
2001-06-26    0.45
2001-06-30    0.65
2001-07-04    0.70
2001-07-08    0.98
2001-07-11    1.01
2001-07-14    1.01
2001-07-15    1.22
2001-07-19    1.31
2001-07-22    1.51
2001-07-29    1.67
2001-08-03    1.77
2001-08-05    1.91
2001-08-11    2.00
2001-08-13    2.05
2001-08-18    2.21
2001-08-22    2.26
2001-08-25    2.37
2001-09-01    2.43
2002-03-16    2.43
2002-03-23    2.43
Name: CONSORCIO\nDE PRODUCT\nORES DE\nFRUTA S.A., dtype: float64
               AGRICOLA\nALPAMAYO\nS.A.  AGRICOLA\nAUSTRAL\nS.A.C  \
FECHA_MOD_FER                                                       
2001-06-01                          0.0                       0.0   
2001-06-04                          0.0                       0.0   
2001-06-10                          0.0                       0.0   
2001-06-16                          0.0                       0.0   
2001-06-24                      

In [57]:
#Análisis para Limpieza de datos
"""#FrecuenciaTexto
frecuencia = df_concatenado["DESC_COM"].value_counts()
#Porcentaje de texto
porcentajeDeFrecuencia = (frecuencia / frecuencia.sum())*100

#Creamos un dataframe con los resultados
resultados = pd.DataFrame({"frecuencia": frecuencia, "Porcentaje": porcentajeDeFrecuencia} )
#Calculamos el porcentaje acumulado
resultados["Porcentaje acumulado"] = resultados["Porcentaje"].cumsum()
#Filtramos los que llegan al 80%

resultados= resultados[resultados["Porcentaje acumulado"]>80]
print(resultados)"""

'#FrecuenciaTexto\nfrecuencia = df_concatenado["DESC_COM"].value_counts()\n#Porcentaje de texto\nporcentajeDeFrecuencia = (frecuencia / frecuencia.sum())*100\n\n#Creamos un dataframe con los resultados\nresultados = pd.DataFrame({"frecuencia": frecuencia, "Porcentaje": porcentajeDeFrecuencia} )\n#Calculamos el porcentaje acumulado\nresultados["Porcentaje acumulado"] = resultados["Porcentaje"].cumsum()\n#Filtramos los que llegan al 80%\n\nresultados= resultados[resultados["Porcentaje acumulado"]>80]\nprint(resultados)'

In [58]:
import bar_chart_race as bcr
import matplotlib.pyplot as plt
# Función para formatear las etiquetas de las barras
def custom_label_func(values, ranks):
    return [f'{val:,.2f}' for val in values]


# Crear el bar chart race
bcr.bar_chart_race(
    df=df_tabla_dinamica.head(25),
    filename='bar_chart_race.mp4',
    orientation='h',
    sort='desc',
    n_bars=10,
    fixed_order=False,
    fixed_max=False,
    steps_per_period=10,
    interpolate_period=False,
    label_bars=True,
    bar_size=.80,
    period_label={'x': .99, 'y': .25, 'ha': 'right', 'va': 'center'},
    period_fmt='%B %d, %Y',
    period_summary_func=lambda v, r: {'x': .99, 'y': .18, 's': f'Total: {v.nlargest(3).sum():,.0f}', 'ha': 'right', 'size': 4},
    perpendicular_bar_func='median',
    period_length=200,
    figsize=(3.5, 7),
    dpi=400,
    cmap='dark12',
    title='Bar Chart Race',
    title_size='',
    bar_label_size=5,
    tick_label_size=7,
    shared_fontdict={'family': 'Verdana', 'color': '.1'},
    scale='linear',
    writer=None,
    fig=None,
    bar_kwargs={'alpha': .7},
    filter_column_colors=False
)


c:\Users\ymnl_\AppData\Local\Programs\Python\Python312\Lib\site-packages\bar_chart_race\_make_chart.py:889: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_values.iloc[:, 0] = df_values.iloc[:, 0].fillna(method='ffill')
c:\Users\ymnl_\AppData\Local\Programs\Python\Python312\Lib\site-packages\bar_chart_race\_make_chart.py:284: UserWarning: Glyph 127 () missing from font(s) Verdana.
  fig.canvas.print_figure(io.BytesIO())
c:\Users\ymnl_\AppData\Local\Programs\Python\Python312\Lib\site-packages\bar_chart_race\_make_chart.py:284: UserWarning: Glyph 128 (\x80) missing from font(s) Verdana.
  fig.canvas.print_figure(io.BytesIO())
c:\Users\ymnl_\AppData\Local\Programs\Python\Python312\Lib\site-packages\bar_chart_race\_make_chart.py:284: UserWarning: Glyph 129 (\x81) missing from font(s) Verdana.
  fig.canvas.print_figure(io.BytesIO())
c:\Users\ymnl_\AppData\Local\Programs\Python\Python312\Lib\site-packa